# Plano de Teste

### Testes na camada trusted para os dados sobre de internet fixa.
Testes realizados após transformar os dataset de formato shapefile em csv


In [12]:
#importando bibliotecas necessárias para os testes
import pandas as pd
import great_expectations as gx 

#Criar o df tax_df com as informacoes doa arquivo us_tax_data_2016.csv
path ='./dados/validacoes_trusted/trusted_speedtest_fixed.csv'
df_fixed = pd.read_csv(path, delimiter=';', encoding="ansi")

In [13]:
# verificacao do layout do dataset
print(df_fixed.head(3))


            quadkey  avg_d_kbps  avg_u_kbps  avg_lat_ms  tests  devices   
0  2110223123011000      157341       56947          13   1095      259  \
1  2110223121232032      184049      107142           4    120       45   
2  2110223123001110      176971       90063          23    221       52   

   index_righ   CD_MUN   latitude  longitude        date  quarter network_ty   
0        3174  3205309 -20.305993 -40.295105  2022-01-01        1      fixed  \
1        3174  3205309 -20.280232 -40.328064  2022-01-01        1      fixed   
2        3174  3205309 -20.305993 -40.350037  2022-01-01        1      fixed   

                                            geometry  
0  POLYGON ((-40.2978515625 -20.3034175184893, -4...  
1  POLYGON ((-40.330810546875 -20.2776560563371, ...  
2  POLYGON ((-40.352783203125 -20.3034175184893, ...  


Crie uma expectation para utilizar no great expectation para relizações de algumas validações:

In [14]:
df_gx = gx.dataset.PandasDataset(df_fixed)
df_gx.head(3)

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,index_righ,CD_MUN,latitude,longitude,date,quarter,network_ty,geometry
0,2110223123011000,157341,56947,13,1095,259,3174,3205309,-20.305993,-40.295105,2022-01-01,1,fixed,"POLYGON ((-40.2978515625 -20.3034175184893, -4..."
1,2110223121232032,184049,107142,4,120,45,3174,3205309,-20.280232,-40.328064,2022-01-01,1,fixed,"POLYGON ((-40.330810546875 -20.2776560563371, ..."
2,2110223123001110,176971,90063,23,221,52,3174,3205309,-20.305993,-40.350037,2022-01-01,1,fixed,"POLYGON ((-40.352783203125 -20.3034175184893, ..."


``1`` Verifica se o código município (CD_MUN) contém apenas valores únicos.

Usar a função ``expect_column_values_to_be_unique`` para definir essa expectativa

In [15]:
validation_result = df_gx.expect_column_values_to_be_unique(column = 'CD_MUN')
validation_result


{
  "meta": {},
  "success": false,
  "result": {
    "element_count": 1807570,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 1807569,
    "unexpected_percent": 99.99994467710795,
    "unexpected_percent_total": 99.99994467710795,
    "unexpected_percent_nonmissing": 99.99994467710795,
    "partial_unexpected_list": [
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309,
      3205309
    ]
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [16]:
if validation_result['success']:
    print('A coluna de código de município contem apenas valores únicos')
else:
    print('A coluna código de município possui valores duplicados')

A coluna código de município possui valores duplicados


``2`` Validação de valores nulos em uma coluna


In [17]:
validation_result = df_gx.expect_column_values_to_not_be_null(column= 'CD_MUN')
validation_result


{
  "meta": {},
  "success": true,
  "result": {
    "element_count": 1807570,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [18]:
if validation_result['success']:
    print('A coluna código do município não possui valores nulos')
else:
    print('A coluna código do município possui valores nulos, verificar a necessidade de realizar tratamento no campo')

A coluna código do município não possui valores nulos


``3`` Verifica se há valores inválidos para latitude.


In [19]:
latitude_invalida = df_gx[(df_gx['latitude'] < -90) | (df_gx['latitude'] > 90)]

if latitude_invalida.empty:
    print("Não há valores incorretos na coluna latitude.")
else:
    print("Existem valores incorretos na coluna latitude:")
    print('Valor(es) incorreto(s):')
    print(latitude_invalida['latitude'].to_string(index=False))

Não há valores incorretos na coluna latitude.


In [20]:
null_latitude_count = latitude_invalida['latitude'].count()
print("Quantidade de valor(es) de latitude incorreto(s):", null_latitude_count)


Quantidade de valor(es) de latitude incorreto(s): 0


In [21]:
longitude_invalida = df_gx[(df_gx['longitude'] < -180) | (df_gx['longitude'] > 180)]

if longitude_invalida.empty:
    print("Não há valores incorretos na coluna longitude.")
else:
    print("Existem valores incorretos na coluna longitude:")
    print('Valor(es) incorreto(s):')
    print(longitude_invalida['longitude'].to_string(index=False))


Não há valores incorretos na coluna longitude.


``4`` Verifica se há data(s) incorreta(s).

In [22]:
from datetime import datetime

data_invalida = []

for valor_data in df_gx['date']:
    try:
        datetime.strptime(valor_data, '%Y-%m-%d')
    except ValueError:
        data_invalida.append(valor_data)

if len(data_invalida) == 0:
    print("Não há valores incorretos na coluna de datas.")
else:
    # Criar DataFrame com os valores incorretos
    df_data_invalida = pd.DataFrame({'Data(s) Incorreta(s):': data_invalida})
    print("Existe(m) valor(es) incorreto(s) na coluna de datas:")
    print(df_data_invalida.to_string(index=False))

Não há valores incorretos na coluna de datas.
